## Statistical Experiments on the data

In [2]:
import pandas
nodes = pandas.read_csv("../nodes_official.csv")
edges = pandas.read_csv("../edges_official.csv")
nodes

,ID,Case,Age,Gender,Nationality,Condition,Transmission,Cluster,Confirmed On,Recovered On,Discharged On,Deceased On
0,MAV00001,MAV00001,49,Female,Turkey,Stable,Local Transmission,NaN,07 March 2020,21 March 2020,23 March 2020,NaN
1,MAV00002,MAV00002,56,Male,South Africa,Stable,Local Transmission,NaN,07 March 2020,23 March 2020,23 March 2020,NaN
2,MAV00003,MAV00003,69,Male,Italy,Stable,Imported,NaN,08 March 2020,20 March 2020,31 March 2020,NaN
3,MAV00004,MAV00004,67,Female,Italy,Stable,Imported,NaN,08 March 2020,20 March 2020,31 March 2020,NaN
4,MAV00005,MAV00005,27,Female,Germany,Stable,Imported,NaN,09 March 2020,23 March 2020,29 March 2020,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
14880,MAV14881,MAV14881,9,Male,Maldives,Stable,Cluster of Cases,HF158102,22 January 2021,NaN,NaN,NaN
14881,MAV14882,MAV14882,30,Female,Maldives,Stable,Cluster of Cases,HF158102,22 January 2021,NaN,NaN,NaN
14882,MAV14883,MAV14883,45,Male,Maldives,Stable,Cluster of Cases,NaN,22 January 2021,NaN,NaN,NaN
14883,MAV14884,MAV14884,29,Female,France,Stable,Cluster of Cases,NaN,23 January 2021,NaN,NaN,NaN


#### Cluster Stats

In [10]:
cluster_counts = nodes['Cluster'].value_counts()
print(cluster_counts[:20])

RFLH        196
Travel      192
COV         133
STC         127
AMH         112
SHEWD1      103
CHEC         95
MND          90
CCC          87
OLHV1        83
OBLU1        82
GABL         79
KDKR         76
POL          72
CRGO         72
LKHD         65
KMMA         62
SGN          56
HP2N3-33     56
LLY          53
Name: Cluster, dtype: int64


In [13]:
cluster_counts = nodes['Age'].value_counts()
print(cluster_counts[:20])

30    476
32    462
28    442
31    441
35    425
25    425
27    413
26    405
33    403
24    396
34    395
29    375
37    360
36    359
23    350
38    346
22    312
40    308
39    261
41    237
Name: Age, dtype: int64
